<figure>
   <IMG SRC="https://mamba-python.nl/images/logo_basis.png" WIDTH=125 ALIGN="right">
</figure>

# Logging
_developed by Onno Ebbens_
    
<hr>

Logging is a tool to help programmers:
- understand the different steps in an application
- save information about the proces
- gain further insights into errors and how they propagate
    
By logging important information it gets easier to debug the code. Alos the save information, the logs, can help to analyse performance and user patterns. Python has a built-in logging module that can be added to your application. This notebook explains how you can use this module.
    
Required prior knowledge:
- know how to define a function, modify a function and call that function
- know what a numpy array is.
      
### Content<a id="top"></a>
1. [Print](#1)
2. [Logging](#2)
3. [Levels](#3)
4. [Log to file](#4)
5. [Log format](#5)
6. [Logger name](#6)   
7. [Advanced logging](#7)   
8. [Answers](#Antwoorden)

## 1. [Print](#top)<a id="1"></a>

The `print` function can be seen as the most basic form of logging. The `print` function can be used in multiple ways. For example to indicate what happens during code execution or to see the value of a variable.

<hr>

#### Exercise 1 <a name="opdr1"></a>

Add the following print statement in the function definition below:
- `print('compute frequency per bin_edge')`
- `print('starting to compute bin_edges')`
- `print('frequency per bin is', count)`
- `print('finished computing bin_edges, bin_edges are:', bin_edges)`
- `print('getting frequency bins of array', arr, 'using', n_bins, 'bins')`

In [3]:
import numpy as np

def frequency_bins(arr, n_bins=4):
    """ get the frequency of the numbers in an array within a number of bins. The boundaries of the bins are 
    created by the percentiles of the array.
    
    Parameters
    ----------
    arr : np.array of ints or float
        numbers to divide into bins
    n_bins : int
        number of bins
    
    Returns
    -------
    count : np.array
        the number of items in arr that are within the bin
    bins : np.array
        the boundaries of the bins
    """
    
    bin_edges = np.zeros(n_bins+1)    
    for i in range(n_bins+1):
        percentile = i*100/n_bins
        bin_edges[i] = np.percentile(arr, percentile)
    
    count, _ = np.histogram(arr,bin_edges)
    
    return count

random_numbers = np.random.randint(0, 10, (10))

count = frequency_bins(random_numbers)

<a href="#antw1">Answer exercise 1</a>
<hr>

In the exercise you could how a print statement can be used to list what happens in the code. However, there are some disadvantages of using a print statement:
- The print statement will always print something, even if you are not interested in that information
- Everytime you call a function with a print statement you see the results only in the console. This can make it harder to find back the information.
- The more print statements you add the more cluttered the results become. It is hard to trace back in which part of the code which print statement is called.

Fortunately Python has another way of displaying the desired information in a structured way in the `logging` module.

## 2. [Logging](#top)<a id="2"></a>

The `logging` module in Python contains functions and classes you can use to keep track of your processes. Using the logging module you can add pieces of text, also called logs, to your code to indicate what is happening. The example below shows how to create a log message.

In [4]:
import logging

# settings for the logging module
logging.basicConfig(level=logging.DEBUG)

# create a log
logging.info('this is a log message')

INFO:root:this is a log message


In the example we create a log message that is shown below the cell. This is very similar to what the `print` function does. A visible difference is the red bar around the text and `INFO:root:` before the log message. The red bar is only shown in Jupyter Notebooks and therefore not a setting of the logging module. `INFO:` refers to the log level and is explained further in the next [chapter](#3). `root:` refers to the logger name and will be explaiend further in [chapter 6](#6).

In the code you also see `logging.basicConfig(..`. This is used to specify the settings in the `logging` module. Some of these settings will be explained in [chapter 3](#3).

<hr>

#### Exercise 2 <a name="opdr2"></a>

Change the `print` statements in the function from exercise 1 into logging statements.

NB: After these change the function contains many logging statements. In reality you will never add that many logging statements to such a small piece of code. For education purposes we will continue to use many logging statements in this notebook.

<a href="#antw2">Answer exercise 2</a>

<hr>

## 3. [Levels](#top)<a id="3"></a>

The logs you create with the `logging` module always have a certain level, the "log level". This level indicates the importance of the log. By giving the logs a level you can choose later at which level you want to show the logs. The `logging` module has the following default levels:
1. DEBUG
2. INFO
3. WARNING
4. ERROR
5. CRITICAL

The logs in level 5 'CRITICAL' are always important while the logs in level 1 'DEBUG' are only important if you want to see a lot of detailed information, e.g. when you are debugging.

In the previous example we've created logs at log level 'INFO'. Below we've created a function with logs on different levels. When calling the function we see the log level at the start of the log message.

In [5]:
def simple_func(shape_a):
    """ function does two things:
    1. create an array a with random integer values between 0 and 10, and shape defind by 'shape_a'
    2. compute array b where each element is the sum of the element in array a and the index number 
    of that element.
    
    Parameters
    ----------
    shape_a : int
        shape of array a
    
    Returns
    -------
    b : np.ndarray
        output array
    """
    logging.debug('start simple_func')
    logging.info(f'shape_a is {shape_a}')
    
    logging.debug('get array a')
    a = np.random.randint(0, 10, shape_a)
    logging.info(f'array a = {a}')

    if a.ndim > 1:
        logging.error(f'expected an array with 1 dimension got {a.ndim}')
        raise ValueError()
    
    if a.shape[0]>1000:
        logging.warning(f'array a has more than 1000 elements computing b can be slow')
    
    logging.debug(f'compute array b from a')
    b = np.ones_like(a)    
    for i, val in enumerate(a):
        b[i] = val + i

    logging.info(f'array b = {b}')
    return b

In [6]:
simple_func(1001)

DEBUG:root:start simple_func
INFO:root:shape_a is 1001
DEBUG:root:get array a
INFO:root:array a = [2 0 7 ... 7 7 8]
DEBUG:root:compute array b from a
INFO:root:array b = [   2    1    9 ... 1005 1006 1008]


array([   2,    1,    9, ..., 1005, 1006, 1008])

In [7]:
simple_func((2,5))

DEBUG:root:start simple_func
INFO:root:shape_a is (2, 5)
DEBUG:root:get array a
INFO:root:array a = [[3 4 8 5 6]
 [3 6 6 9 9]]
ERROR:root:expected an array with 1 dimension got 2


ValueError: 

#### set log level

Je kan kiezen welke logs worden weergegeven door het log level in te stellen:
```
logging.getLogger().setLevel(logging.<log_level>)
```
Vervang in bovenstaande code `<log_level>` door de naam van het log level, bijv. `INFO`. Nadat de instellingen zijn aangepast worden enkel nog de logs van het gekozen level en hoger getoond. In onderstaande voorbeelden kan je zien welke logs worden getoond bij verschillende log levels. 

In [8]:
logging.getLogger().setLevel(logging.INFO)
simple_func(1001)

INFO:root:shape_a is 1001
INFO:root:array a = [6 5 6 ... 6 3 4]
INFO:root:array b = [   6    6    8 ... 1004 1002 1004]


array([   6,    6,    8, ..., 1004, 1002, 1004])

In [9]:
logging.getLogger().setLevel(logging.ERROR)
simple_func(1001)

array([   6,    7,    8, ...,  999, 1005, 1009])

In [10]:
logging.getLogger().setLevel(logging.ERROR)
simple_func((2,5))

ERROR:root:expected an array with 1 dimension got 2


ValueError: 

<hr>

#### Exercise 3 <a name="opdr3"></a>

Change the log levels in the function from exercise 1 & 2. Choose an appropiate log level. Check the logs by calling the function.

<a href="#antw3">Answer exercise 3</a>

<hr>

#### Exercise 4<a name="opdr4"></a>

Use the function from exercise 3, add a warning if the function argument is not a numpy array. You can use this stackoverflow post to see how to do the numpy type checking: https://stackoverflow.com/questions/59279803/how-to-check-if-an-object-is-an-np-arrayb

<a href="#antw4">Answer Exercise 4</a>

<hr>

## 4. [log to file](#top)<a id="4"></a>

Both the `print` function and the `logging` module write the logs to the consule, which is shown at the output of every Jupyter Notebook cell. With the `logging` module it is easy to write the logs to a file instead. We can define this in the `basicConfig` settings from [chapter 2](#2).

Once the `basicConfig` is set we cannot change the settings anymore. This is on purpose to prevent other packages changing your log module settings. When you do want to modify the settings you should restart the kernel and reload the `logging` module. Below we add a neat trick by using the `reload` function. When we reload the `logging` module we can reset the `basicConfig` settings. When you are going to use the `logging` module yourself it is adviced not to use the `reload` function.

In [14]:
from importlib import reload
reload(logging)
logging.basicConfig(filename='example.log', level=logging.DEBUG)

After runnign the code the file `example.log` is created in the same directory as this notebook. If you open the file with a text editor you will see it is still empty.

In [15]:
logging.info('this is the log text and is written to the log file')

When you run the code above and reopen the `example.log` file you will see this line in the file:
```
INFO:root:this is the log text and is written to the log file
```
Every next log message is automatically written to the same file.

In [16]:
logging.info('this line will also appear in the log file')

After running the cel above the file will look like this:
```
INFO:root:this is the log text and is written to the log file
INFO:root:this line will also appear in the log file
```

Watch out! If you run the same code twice the log message will be written twice to the `example.log` file.

<hr>

#### Exercise 5<a name="opdr5"></a>

Create your own log file and call the function `simple_func` from [chapter 3](#3). Make sure the 'DEBUG' logs are not written to the same file.

<a href="#antw5">Answer exercise 5</a>

<hr>

### filemode

After choosing a filename you can also specify in the `basicConfig` what happens when the log file already exists. This is specified with the `filemode` argument. The default settings for `filemode` is `'a'` (append), this implies that the existing log file is kept and new logs are appended to the existing. It is also possible to choose `filemode='w'` (write). Then the existing log file is replaced with a new empty log file so only new logs are shown. See two examples below with the filemodes 'w' en 'a'.

In [18]:
from importlib import reload
reload(logging)
logging.basicConfig(filename='example.log', filemode='w', level=logging.DEBUG)

In [19]:
logging.info('This is the first log in the example.log file')


In [20]:
from importlib import reload
reload(logging)
logging.basicConfig(filename='example.log', filemode='a', level=logging.DEBUG)

In [21]:
logging.info('This is the second line in the example.log file')

## 5. [Log format](#top)<a id="5"></a>

In the `basicConfig` we can also set the log format. We can choose to add a date, time, log level and logger name to each log. Below some examples on how to add this information.

In [22]:
import logging
logging.basicConfig(format='%(levelname)s %(asctime)s %(message)s', 
                    datefmt='%Y-%m-%d %I:%M:%S %p',
                    level=logging.DEBUG)
logging.info('this is my log message')

In [23]:
# voeg een datumtijd toe aan het log bericht
from importlib import reload
reload(logging)
logging.basicConfig(format='%(asctime)s %(message)s', 
                    datefmt='%Y-%m-%d %I:%M:%S %p')
logging.warning('is the time this message was logged')

2022-09-14 10:09:32 PM is the time this message was logged


In [24]:
# voeg het log level (levelname) toe aan het log bericht
from importlib import reload
reload(logging)
logging.basicConfig(format='%(levelname)s %(message)s')
logging.warning('<- this is my log level')
logging.critical('<- this is my log level')

WARNING <- this is my log level
CRITICAL <- this is my log level


<hr>

#### Exercise 6<a name="opdr6"></a>

Make sure the log messages from `simple_func` (see [chapter 3](#3)) are written to the file with the following information:
- the time of the log in format HH:MM:SS
- the log level

<a href="#antw5">Answer exercise 6</a>

<hr>

## 6. [Logger name](#top)<a id="6"></a>

Sometimes it is usefull to know which package, module or function generated a certain log message. This is also possible with the logging module by creating multiple loggers. We do this in three steps:
1. define a logger with logger name: `l = logging.getLogger('<logger name>')`
2. create logs using the logger using `l.info('mijn log')` instead of the previously used `logging.info('mijn log')`
3. modify the format in the `basicConfig` settings in such a way that the logger name is shown in the log message.

Below we have two functions with examples.

When you do not define a logger name en use `logging.info()` the logger name is automatically set to 'root'.

In [25]:
def remove_exclamation_mark(s):
    l = logging.getLogger('function->remove_exclamation_mark')
    l.debug('run function remove_exclamation_mark')
    
    l.debug('check if input string has exclamation_marks')
    if '!' in s:
        l.info('input string contains exclamation_marks')
    else:
        l.warning('input string has no exclamation_marks!')
    
    l.info('trying to remove exclamation mark')
    new_s = s.replace('!', '')
    
    return new_s
    
def replace_space_with_underscore(s):
    l = logging.getLogger('function->replace_space_with_underscore')
    l.debug('run function replace_space_with_underscore')
    
    l.debug('check if input string has spaces')
    if ' ' in s:
        l.info('input string contains one or more spaces')
    else:
        l.warning('input string has no spaces!')
    
    l.info('trying to replace spaces with underscores')
    new_s = s.replace(' ', '_')
    
    return new_s  
    
# voeg de logger naam toe aan basicConfig
from importlib import reload
reload(logging)
logging.basicConfig(format='%(name)s: %(levelname)s: %(message)s', datefmt='%I:%M:%S',
                    level=logging.DEBUG)

logging.info('testing the logger names per function')

s = 'test string with spaces and an exclamation mark!'
new_s = remove_exclamation_mark(s)
new_s = replace_space_with_underscore(new_s)


s = 'test_string_without_spaces!'
new_s = remove_exclamation_mark(s)
new_s = replace_space_with_underscore(new_s)

root: INFO: testing the logger names per function
function->remove_exclamation_mark: DEBUG: run function remove_exclamation_mark
function->remove_exclamation_mark: DEBUG: check if input string has exclamation_marks
function->remove_exclamation_mark: INFO: input string contains exclamation_marks
function->remove_exclamation_mark: INFO: trying to remove exclamation mark
function->replace_space_with_underscore: DEBUG: run function replace_space_with_underscore
function->replace_space_with_underscore: DEBUG: check if input string has spaces
function->replace_space_with_underscore: INFO: input string contains one or more spaces
function->replace_space_with_underscore: INFO: trying to replace spaces with underscores
function->remove_exclamation_mark: DEBUG: run function remove_exclamation_mark
function->remove_exclamation_mark: DEBUG: check if input string has exclamation_marks
function->remove_exclamation_mark: INFO: input string contains exclamation_marks
function->remove_exclamation_mark:

NB: It is very uncommon to define seperate logger names for each function. There are often seperate loggers defined per module. The logger name is often set to the `__name__` variable that points to the name of the module.

In [26]:
reload(logging)
logging.basicConfig(level=logging.DEBUG)

l = logging.getLogger(__name__)
l.info('this is a log from the notebook')
import example_module
example_module.check_if_nummerical(6)

INFO:__main__:this is a log from the notebook
INFO:example_module:running check_if_nummerical function inside the example_module


True

## 7. [Advanced](#top)<a id="7"></a>

The logging module contains multiple possibilities to keep track of the logs. You can for example:
- keep multiple log files to order different log messages
- instead of creating a file sent the logs via an e-mail
- create a seperate files with all the logger settings

For more advanced logging features a [tutorial](https://docs.python.org/3/howto/logging.html#logging-advanced-tutorial) is available. More options for formatting you logs are shown [here](https://docs.python.org/3/library/logging.html#logrecord-attributes).

## [Answers](#top)<a id="Antwoorden"></a>

<hr>

#### <a href="#opdr1">Answer exercise 1</a> <a name="antw1"></a>



In [27]:
import numpy as np

def frequency_bins(arr, n_bins=4):
    """ get the frequency of the numbers in an array within a number of bins. The boundaries of the bins are 
    created by the percentiles of the array.
    
    Parameters
    ----------
    arr : np.array of ints or float
        numbers to divide into bins
    n_bins : int
        number of bins
    
    Returns
    -------
    count : np.array
        the number of items in arr that are within the bin
    bins : np.array
        the boundaries of the bins
    """
    print(f'getting frequency bins of array {arr} using {n_bins} bins')
    
    print('starting to compute bin_edges')
    
    bin_edges = np.zeros(n_bins+1)    
    for i in range(n_bins+1):
        percentile = i*100/n_bins
        bin_edges[i] = np.percentile(arr, percentile)
        
    print(f'finished computing bin_edges, bin_edges are: {bin_edges}')
    
    print('compute frequency per bin_edge')
    
    count, _ = np.histogram(arr,bin_edges)
    
    print(f'frequency per bin is {count}')
    
    return count

random_numbers = np.random.randint(0, 10, (10))

count = frequency_bins(random_numbers)

getting frequency bins of array [7 3 4 0 9 6 3 5 5 7] using 4 bins
starting to compute bin_edges
finished computing bin_edges, bin_edges are: [0.   3.25 5.   6.75 9.  ]
compute frequency per bin_edge
frequency per bin is [3 1 3 3]


<hr>

#### <a href="#opdr2">Answer exercise 2</a> <a name="antw2"></a>

See function below

In [28]:
import numpy as np

def frequency_bins(arr, n_bins=4):
    """ get the frequency of the numbers in an array within a number of bins. The boundaries of the bins are 
    created by the percentiles of the array.
    
    Parameters
    ----------
    arr : np.array of ints or float
        numbers to divide into bins
    n_bins : int
        number of bins
    
    Returns
    -------
    count : np.array
        the number of items in arr that are within the bin
    bins : np.array
        the boundaries of the bins
    """
    logging.info(f'getting frequency bins of array {arr} using {n_bins} bins')
    
    logging.info('starting to compute bin_edges')
    
    bin_edges = np.zeros(n_bins+1)    
    for i in range(n_bins+1):
        percentile = i*100/n_bins
        bin_edges[i] = np.percentile(arr, percentile)
        
    logging.info(f'finished computing bin_edges, bin_edges are: {bin_edges}')
    
    logging.info('compute frequency per bin_edge')
    
    count, _ = np.histogram(arr,bin_edges)
    
    logging.info(f'frequency per bin is {count}')
    
    return count

random_numbers = np.random.randint(0, 10, (10))

count = frequency_bins(random_numbers)

INFO:root:getting frequency bins of array [5 4 6 8 3 1 1 3 5 7] using 4 bins
INFO:root:starting to compute bin_edges
INFO:root:finished computing bin_edges, bin_edges are: [1.   3.   4.5  5.75 8.  ]
INFO:root:compute frequency per bin_edge
INFO:root:frequency per bin is [2 3 2 3]


<hr>

#### <a href="#opdr3">Answer exercise 3</a> <a name="antw3"></a>

See the function below and calling the function in the cells below.

In [29]:
import numpy as np

def frequency_bins(arr, n_bins=4):
    """ get the frequency of the numbers in an array within a number of bins. The boundaries of the bins are 
    created by the percentiles of the array.
    
    Parameters
    ----------
    arr : np.array of ints or float
        numbers to divide into bins
    n_bins : int
        number of bins
    
    Returns
    -------
    count : np.array
        the number of items in arr that are within the bin
    bins : np.array
        the boundaries of the bins
    """
    logging.info(f'getting frequency bins of array {arr} using {n_bins} bins')
    
    logging.debug('starting to compute bin_edges')
    
    bin_edges = np.zeros(n_bins+1)    
    for i in range(n_bins+1):
        percentile = i*100/n_bins
        bin_edges[i] = np.percentile(arr, percentile)
        
    logging.info(f'finished computing bin_edges, bin_edges are: {bin_edges}')
    
    logging.debug('compute frequency per bin_edge')
    
    count, _ = np.histogram(arr,bin_edges)
    
    logging.info(f'frequency per bin is {count}')
    
    return count

In [30]:
logging.getLogger().setLevel(logging.DEBUG)

random_numbers = np.random.randint(0, 10, (10))
count = frequency_bins(random_numbers)

INFO:root:getting frequency bins of array [3 4 9 5 9 3 5 0 7 0] using 4 bins
DEBUG:root:starting to compute bin_edges
INFO:root:finished computing bin_edges, bin_edges are: [0.  3.  4.5 6.5 9. ]
DEBUG:root:compute frequency per bin_edge
INFO:root:frequency per bin is [2 3 2 3]


In [31]:
logging.getLogger().setLevel(logging.INFO)
random_numbers = np.random.randint(0, 10, (10))
count = frequency_bins(random_numbers)

INFO:root:getting frequency bins of array [1 5 2 6 2 9 5 6 4 6] using 4 bins
INFO:root:finished computing bin_edges, bin_edges are: [1.  2.5 5.  6.  9. ]
INFO:root:frequency per bin is [3 1 2 4]


<hr>

#### <a href="#opdr4">Answer exercise 4</a> <a name="antw4"></a>

See the function below.

In [32]:
import numpy as np

def frequency_bins(arr, n_bins=4):
    """ get the frequency of the numbers in an array within a number of bins. The boundaries of the bins are 
    created by the percentiles of the array.
    
    Parameters
    ----------
    arr : np.array of ints or float
        numbers to divide into bins
    n_bins : int
        number of bins
    
    Returns
    -------
    count : np.array
        the number of items in arr that are within the bin
    bins : np.array
        the boundaries of the bins
    """
    if not isinstance(arr, np.ndarray):
        logging.warning(f'Unexpected type of argument arr. Type is {type(arr)}, expected type {np.ndarray}.')
    
    
    logging.info(f'getting frequency bins of array {arr} using {n_bins} bins')
    
    logging.debug('starting to compute bin_edges')
    
    bin_edges = np.zeros(n_bins+1)    
    for i in range(n_bins+1):
        percentile = i*100/n_bins
        bin_edges[i] = np.percentile(arr, percentile)
        
    logging.info(f'finished computing bin_edges, bin_edges are: {bin_edges}')
    
    logging.debug('compute frequency per bin_edge')
    
    count, _ = np.histogram(arr,bin_edges)
    
    logging.info(f'frequency per bin is {count}')
    
    return count

In [33]:
# check of warning zichtbaar is
logging.getLogger().setLevel(logging.WARNING)
random_numbers = np.random.randint(0, 10, (10))
count = frequency_bins(random_numbers.tolist())

<hr>

#### <a href="#opdr5">Answer exercise 5</a> <a name="antw5"></a>

See the code below, make sure to:
- use `reload` to reload the `logging` module so you can change the settings of `basicConfig`.
- It is not necesary to call `logging.getLogger().setLevel` to set the log level. The log level can be set directly in the `basicConfig` using `level=logging.INFO`.

In [34]:
from importlib import reload
reload(logging)

# definieer basicConfig
logging.basicConfig(filename='my_log.txt', level=logging.INFO)

# roep functie aan
simple_func(1001)

array([   7,    3,    3, ..., 1000, 1001, 1004])

<hr>

#### <a href="#opdr6">Answer exercise 6</a> <a name="antw6"></a>


In [36]:
# add the log level (levelname)
from importlib import reload
reload(logging)

# set basicConfig
logging.basicConfig(format='%(asctime)s %(levelname)s %(message)s', datefmt='%H:%M:%S',
                    filename='simple_func.log', filemode='w', level=logging.INFO)

# call function
simple_func(1001)

array([   9,    1,   11, ..., 1000, 1008, 1000])

## Acknowledgement

the following sources were used to create this notebook:
- https://docs.python.org/3/library/logging.html
